In [1]:
# 필요한 라이브러리 설치 및 설정

# 구글 번역 라이브러리 설치
!pip install -q googletrans==4.0.0rc1
!pip install -q pandas tqdm

print("라이브러리 설치 완료!")

import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import time
import re
from typing import List, Dict, Tuple

# 구글 번역 라이브러리
from googletrans import Translator, LANGUAGES

# 번역기 초기화
translator = Translator()

print("구글 번역기 초기화 완료!")
print(f"지원 언어: {len(LANGUAGES)}개")
print("주요 언어:", {k: v for k, v in list(LANGUAGES.items())[:10]})


  DEPRECATION: Building 'googletrans' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'googletrans'. Discussion can be found at https://github.com/pypa/pip/issues/6334


라이브러리 설치 완료!
구글 번역기 초기화 완료!
지원 언어: 107개
주요 언어: {'af': 'afrikaans', 'sq': 'albanian', 'am': 'amharic', 'ar': 'arabic', 'hy': 'armenian', 'az': 'azerbaijani', 'eu': 'basque', 'be': 'belarusian', 'bn': 'bengali', 'bs': 'bosnian'}


In [2]:
# 구글 번역 함수 정의

def detect_language(text: str) -> str:
    """텍스트의 언어를 감지합니다."""
    try:
        # 한국어 문자 비율 체크
        korean_chars = sum(1 for char in text if '\uac00' <= char <= '\ud7af')
        if korean_chars > len(text) * 0.1:  # 10% 이상이 한글이면 한국어
            return "ko"
        
        # 구글 번역으로 언어 감지
        detected = translator.detect(text[:500])  # 처음 500자만 검사
        return detected.lang
        
    except Exception as e:
        print(f"언어 감지 오류: {e}")
        # 중국어/영어 휴리스틱 감지
        chinese_chars = sum(1 for char in text if '\u4e00' <= char <= '\u9fff')
        english_chars = sum(1 for char in text if char.isascii() and char.isalpha())
        
        if chinese_chars > english_chars:
            return "zh"
        else:
            return "en"

def translate_with_google(text: str, target_lang: str = "ko", max_retries: int = 3) -> str:
    """구글 번역으로 텍스트를 번역합니다."""
    
    if not text or len(text.strip()) < 2:
        return text
    
    # 언어 감지
    source_lang = detect_language(text)
    
    # 이미 한국어면 번역하지 않음
    if source_lang == "ko":
        return text
    
        # 번역 시도 (재시도 로직 포함)
    for attempt in range(max_retries):
        try:
            # 텍스트가 너무 길면 분할 (5000자 제한)
            if len(text) > 4000:
                # 문장 단위로 분할
                sentences = re.split(r'[.!?]\s+', text)
                translated_parts = []
                
                for sentence in sentences:
                    if sentence.strip():
                        result = translator.translate(sentence.strip(), dest=target_lang, src=source_lang)
                        translated_parts.append(result.text)
                        time.sleep(0.1)  # API 제한 방지
                
                return ' '.join(translated_parts)
            else:
                # 일반 번역
                result = translator.translate(text, dest=target_lang, src=source_lang)
                return result.text
                
        except Exception as e:
            print(f"번역 시도 {attempt + 1} 실패: {e}")
            if attempt < max_retries - 1:
                time.sleep(1)  # 재시도 전 대기
            else:
                print(f"번역 포기: {text[:50]}...")
                return text  # 원문 반환

def translate_dataset_google(df, output_file, batch_size=20, test_limit=None, delay=0.05):
    """
    구글 번역을 사용하여 데이터셋을 배치별로 번역하고 파일에 저장합니다.
    
    Args:
        df: 원본 데이터프레임
        output_file: 저장할 파일 경로
        batch_size: 배치 크기
        test_limit: 테스트용 데이터 개수 제한
        delay: API 호출 간 지연시간 (초)
    """
    
    # 테스트용으로 데이터 제한
    if test_limit and len(df) > test_limit:
        df = df.head(test_limit)
        print(f"테스트용으로 {test_limit}개 데이터로 제한")
    
    print(f"총 {len(df)}개 항목을 {batch_size}개씩 배치 번역 시작...")
    print(f"결과 파일: {output_file}")
    
    # 파일 헤더 작성 (덮어쓰기)
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write("Question,Answer\n")
    
    total_processed = 0
    start_time = time.time()
    
    for batch_start in tqdm(range(0, len(df), batch_size), desc="🌐 구글 번역"):
        batch_end = min(batch_start + batch_size, len(df))
        batch_df = df.iloc[batch_start:batch_end]
        
        batch_results = []
        
        for idx, row in batch_df.iterrows():
            try:
                # Question과 Answer 각각 번역
                original_question = str(row['Question'])
                original_answer = str(row['Answer'])
                
                # 번역 수행
                translated_question = translate_with_google(original_question)
                time.sleep(delay)  # API 제한 방지
                
                translated_answer = translate_with_google(original_answer)
                time.sleep(delay)  # API 제한 방지
                
                batch_results.append({
                    'Question': translated_question,
                    'Answer': translated_answer
                })
                
            except Exception as e:
                print(f"번역 오류 (행 {idx}): {e}")
                batch_results.append({
                    'Question': original_question,
                    'Answer': original_answer
                })
        
        # 배치 결과를 파일에 추가 (append 모드)
        with open(output_file, 'a', encoding='utf-8') as f:
            for result in batch_results:
                # CSV 형식으로 저장 (따옴표 이스케이프)
                question_escaped = result['Question'].replace('"', '""').replace('\n', '\\n')
                answer_escaped = result['Answer'].replace('"', '""').replace('\n', '\\n')
                f.write(f'"{question_escaped}","{answer_escaped}"\n')
        
        total_processed += len(batch_results)
        
        # 진행률 및 예상 시간 표시
        elapsed_time = time.time() - start_time
        avg_time_per_item = elapsed_time / total_processed if total_processed > 0 else 0
        remaining_items = len(df) - total_processed
        estimated_remaining = avg_time_per_item * remaining_items
        
        print(f"배치 {batch_start//batch_size + 1} 완료: {total_processed}/{len(df)}개 처리됨 "
              f"(평균 {avg_time_per_item:.1f}초/개, 예상 남은시간: {estimated_remaining/60:.1f}분)")
    
    total_time = time.time() - start_time
    print(f"🎉 번역 완료! 총 {total_processed}개 항목, 소요시간: {total_time/60:.1f}분")
    return total_processed

print("구글 번역 함수 정의 완료!")


구글 번역 함수 정의 완료!


In [3]:
# 데이터 로드

print("=== 데이터 로드 ===")

# 1. CyberMetric MCQA (영어 전용)
cyber_mcqa = pd.read_csv('../data/CyberMetric/mcqa_org.csv')
print(f"CyberMetric MCQA: {cyber_mcqa.shape[0]}개")
print("샘플:", cyber_mcqa.iloc[0]['Question'][:100] + "...")

# 2. SecBench MCQA (영어/중국어 혼용)
sec_mcqa = pd.read_csv('../data/SecBench/mcqa_org.csv')
print(f"SecBench MCQA: {sec_mcqa.shape[0]}개")
print("샘플:", sec_mcqa.iloc[0]['Question'][:100] + "...")

# 3. SecBench QA (영어/중국어 혼용)  
sec_qa = pd.read_csv('../data/SecBench/qa_org.csv')
print(f"SecBench QA: {sec_qa.shape[0]}개")
print("샘플:", sec_qa.iloc[0]['Question'][:100] + "...")

print("\n데이터 로드 완료!")

# 언어 분포 간단 체크
print("\n=== 언어 분포 체크 ===")
for name, df in [("CyberMetric", cyber_mcqa), ("SecBench MCQA", sec_mcqa), ("SecBench QA", sec_qa)]:
    sample_text = df.iloc[0]['Question']
    detected_lang = detect_language(sample_text)
    print(f"{name}: {detected_lang} 감지됨")


=== 데이터 로드 ===
CyberMetric MCQA: 10180개
샘플: Which of the following refers to the secrecy of information?
1. Availability
2. Authentication
3. In...
SecBench MCQA: 2730개
샘플: 以下哪种措施是企业为了遵守数据保护法规，通常会实施的？
1. 定期进行网络渗透测试
2. 建立数据最小化原则，仅收集必要的个人数据
3. 在网站上使用不可跟踪的cookies
4. 完全避免跨境数据传...
SecBench QA: 270개
샘플: 为什么现代操作系统的IP ID通常是随机产生的？...

데이터 로드 완료!

=== 언어 분포 체크 ===
CyberMetric: en 감지됨
SecBench MCQA: zh-CN 감지됨
SecBench QA: zh-CN 감지됨


In [5]:
# 전체 데이터 번역 실행 - 최대 속도 설정

print("🚀 전체 데이터 구글 번역 시작!")
print("✅ 최적화 설정:")
print("   - 배치 크기: 20개 (속도 향상)")
print("   - API 지연: 0.05초 (최소 지연)")
print("   - 테스트 제한: 없음 (전체 데이터)")
print("   - 재시도: 3회 (안정성 확보)")

# 1. CyberMetric MCQA 번역 (영어 → 한국어)
print(f"\n=== 1. CyberMetric MCQA 번역 ({len(cyber_mcqa)}개) ===")
cybermetric_output = "../data/CyberMetric/mcqa.csv"
translate_dataset_google(
    df=cyber_mcqa,
    output_file=cybermetric_output,
    batch_size=20,      # 속도 향상
    test_limit=None,    # 전체 데이터
    delay=0.05          # 최소 지연
)

# 2. SecBench MCQA 번역 (중국어/영어 → 한국어)
print(f"\n=== 2. SecBench MCQA 번역 ({len(sec_mcqa)}개) ===")
secbench_mcqa_output = "../data/SecBench/mcqa.csv"
translate_dataset_google(
    df=sec_mcqa,
    output_file=secbench_mcqa_output,
    batch_size=20,      # 속도 향상
    test_limit=None,    # 전체 데이터
    delay=0.05          # 최소 지연
)

# 3. SecBench QA 번역 (중국어/영어 → 한국어)
print(f"\n=== 3. SecBench QA 번역 ({len(sec_qa)}개) ===")
secbench_qa_output = "../data/SecBench/qa.csv"
translate_dataset_google(
    df=sec_qa,
    output_file=secbench_qa_output,
    batch_size=20,      # 속도 향상
    test_limit=None,    # 전체 데이터
    delay=0.05          # 최소 지연
)

print("\n🎉 모든 번역 작업 완료!")
print("\n📁 생성된 파일:")
print(f"- CyberMetric MCQA: {cybermetric_output}")
print(f"- SecBench MCQA: {secbench_mcqa_output}")
print(f"- SecBench QA: {secbench_qa_output}")

# 예상 시간 계산
total_items = len(cyber_mcqa) + len(sec_mcqa) + len(sec_qa)
estimated_minutes = total_items * 0.1 / 60  # 0.1초/개 기준
print(f"\n⏱️ 예상 총 소요시간: {estimated_minutes:.1f}분")
print("💡 구글 번역 API 무료 할당량: 월 50만 문자")


🚀 전체 데이터 구글 번역 시작!
✅ 최적화 설정:
   - 배치 크기: 20개 (속도 향상)
   - API 지연: 0.05초 (최소 지연)
   - 테스트 제한: 없음 (전체 데이터)
   - 재시도: 3회 (안정성 확보)

=== 1. CyberMetric MCQA 번역 (10180개) ===
총 10180개 항목을 20개씩 배치 번역 시작...
결과 파일: ../data/CyberMetric/mcqa.csv


🌐 구글 번역:   0%|          | 0/509 [00:00<?, ?it/s]

🌐 구글 번역:   0%|          | 1/509 [00:25<3:37:55, 25.74s/it]

배치 1 완료: 20/10180개 처리됨 (평균 1.3초/개, 예상 남은시간: 218.2분)


🌐 구글 번역:   0%|          | 2/509 [00:54<3:53:04, 27.58s/it]

...

배치 12 완료: 240/270개 처리됨 (평균 5.2초/개, 예상 남은시간: 2.6분)


🌐 구글 번역:  93%|█████████▎| 13/14 [22:16<01:41, 101.80s/it]

배치 13 완료: 260/270개 처리됨 (평균 5.1초/개, 예상 남은시간: 0.9분)


🌐 구글 번역: 100%|██████████| 14/14 [23:08<00:00, 99.15s/it] 

배치 14 완료: 270/270개 처리됨 (평균 5.1초/개, 예상 남은시간: 0.0분)
🎉 번역 완료! 총 270개 항목, 소요시간: 23.1분

🎉 모든 번역 작업 완료!

📁 생성된 파일:
- CyberMetric MCQA: ../data/CyberMetric/mcqa.csv
- SecBench MCQA: ../data/SecBench/mcqa.csv
- SecBench QA: ../data/SecBench/qa.csv

⏱️ 예상 총 소요시간: 22.0분
💡 구글 번역 API 무료 할당량: 월 50만 문자


In [7]:
# 번역 결과 확인

def check_translation_results():
    """번역 결과 파일들을 확인합니다."""
    
    files_to_check = [
        ("../data/CyberMetric/mcqa.csv", "CyberMetric MCQA"),
        ("../data/SecBench/mcqa.csv", "SecBench MCQA"),
        ("../data/SecBench/qa.csv", "SecBench QA")
    ]
    
    print("=== 번역 결과 확인 ===")
    
    for file_path, name in files_to_check:
        if os.path.exists(file_path):
            try:
                df = pd.read_csv(file_path)
                print(f"\n✅ {name}")
                print(f"   파일: {file_path}")
                print(f"   행 수: {len(df)}개")
                print(f"   컬럼: {list(df.columns)}")
                
                # 첫 번째 번역 결과 샘플 표시
                if len(df) > 0:
                    print(f"   번역 샘플:")
                    print(f"   Q: {df.iloc[0]['Question'][:80]}...")
                    print(f"   A: {df.iloc[0]['Answer'][:80]}...")
                    
            except Exception as e:
                print(f"❌ {name}: 파일 읽기 오류 - {e}")
        else:
            print(f"❌ {name}: 파일 없음 - {file_path}")
    
    print("\n📊 번역 품질 확인을 위해 원본과 비교해보세요!")

# 결과 확인 실행
check_translation_results()


=== 번역 결과 확인 ===

✅ CyberMetric MCQA
   파일: ../data/CyberMetric/mcqa.csv
   행 수: 10180개
   컬럼: ['Question', 'Answer']
   번역 샘플:
   Q: 다음 중 정보의 비밀을 나타내는 것은 무엇입니까?\n1. 가용성\n2. 인증\n3. 성실성\n4. 기밀성...
   A: 답변: 4...

✅ SecBench MCQA
   파일: ../data/SecBench/mcqa.csv
   행 수: 2730개
   컬럼: ['Question', 'Answer']
   번역 샘플:
   Q: 다음 중 데이터 보호 규정을 준수하기 위해 기업이 일반적으로 구현하는 조치는 무엇입니까?\n1. 정기적 인 네트워크 침투 테스트\n2. 데이터 ...
   A: 답변: 2...

✅ SecBench QA
   파일: ../data/SecBench/qa.csv
   행 수: 270개
   컬럼: ['Question', 'Answer']
   번역 샘플:
   Q: 현대 운영 체제의 IP ID가 일반적으로 무작위로 생성되는 이유는 무엇입니까?...
   A: 보안을 개선하고 특정 유형의 사이버 공격을 방지합니다....

📊 번역 품질 확인을 위해 원본과 비교해보세요!


In [4]:
# MCQA 답변 형태 개선 - "답변: 번호, 선지" 형태로 변경
# QLoRA 학습 개선

import re
import pandas as pd
from tqdm import tqdm
import os

def extract_choices_from_question(question_text):
    """질문에서 선택지들을 추출합니다."""
    choices = {}
    
    # \n 문자를 실제 개행으로 변환 (CyberMetric, SecBench 대응)
    question_text = question_text.replace('\\n', '\n')
    
    # 질문 텍스트를 줄 단위로 분할
    lines = question_text.split('\n')
    
    for line in lines:
        line = line.strip()
        # "번호. 내용" 패턴 매칭
        match = re.match(r'^(\d+)\.\s*(.+)', line)
        if match:
            number = match.group(1)
            content = match.group(2).strip()
            choices[number] = content
    
    return choices

def enhance_mcqa_answer(question, answer):
    """MCQA 답변을 '답변: 번호, 선지' 형태로 개선합니다."""
    
    # 기존 답변에서 번호 추출 (예: "답변: 1" -> "1")
    answer_match = re.search(r'답변:\s*(\d+)', answer)
    if not answer_match:
        return answer  # 답변 번호를 찾을 수 없으면 원본 반환
    
    answer_number = answer_match.group(1)
    
    # 질문에서 선택지들 추출
    choices = extract_choices_from_question(question)
    
    # 해당 번호의 선택지 찾기
    if answer_number in choices:
        choice_text = choices[answer_number]
        enhanced_answer = f"답변: {answer_number}, {choice_text}"
        return enhanced_answer
    else:
        return answer  # 해당 번호의 선택지를 찾을 수 없으면 원본 반환

def process_mcqa_file(input_file, output_file):
    """MCQA 파일을 처리하여 답변을 개선합니다."""
    
    print(f"📝 {input_file} 처리 중...")
    
    # 데이터 읽기
    df = pd.read_csv(input_file)
    
    print(f"   총 {len(df)}개 항목 발견")
    
    # 답변 개선
    enhanced_rows = []
    
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="답변 개선"):
        question = str(row['Question'])
        answer = str(row['Answer'])
        
        # 답변 개선
        enhanced_answer = enhance_mcqa_answer(question, answer)
        
        enhanced_rows.append({
            'Question': question,
            'Answer': enhanced_answer
        })
    
    # 결과를 새 파일로 저장
    result_df = pd.DataFrame(enhanced_rows)
    result_df.to_csv(output_file, index=False, encoding='utf-8')
    
    print(f"✅ 처리 완료: {output_file}")
    print(f"   {len(result_df)}개 항목이 저장됨")
    
    # 몇 가지 예시 출력
    print("\n📋 변경 예시:")
    for i in range(min(3, len(result_df))):
        original_answer = df.iloc[i]['Answer']
        enhanced_answer = result_df.iloc[i]['Answer']
        if original_answer != enhanced_answer:
            print(f"   원본: {original_answer}")
            print(f"   개선: {enhanced_answer}")
            print()

print("=== MCQA 답변 형태 개선 시작 (수정된 버전) ===")

# 처리할 MCQA 파일들
mcqa_files = [
    ("../data/CyberMetric/mcqa.csv", "../data/CyberMetric/mcqa_enhanced.csv"),
    ("../data/SecBench/mcqa.csv", "../data/SecBench/mcqa_enhanced.csv"),
    ("../data/FinShibainu/mcqa.csv", "../data/FinShibainu/mcqa_enhanced.csv")
]

# 기존 enhanced 파일들 삭제 (다시 생성하기 위해)
for input_file, output_file in mcqa_files:
    if os.path.exists(output_file):
        os.remove(output_file)
        print(f"🗑️ 기존 파일 삭제: {output_file}")

# 함수 테스트
print("\n🧪 함수 테스트:")

# FinShibainu 형태 테스트
test_question1 = """다음 중 고객에게 AMA를 선택할 만한 이유로 적절치 않은 것은?
1. 기존 대출의 만기 연장
2. 높은 이자율의 혜택
3. 조건에 따른 자동 관리
4. 기본계좌와 연동"""

# CyberMetric/SecBench 형태 테스트
test_question2 = "다음 중 정보의 비밀을 나타내는 것은 무엇입니까?\\n1. 가용성\\n2. 인증\\n3. 성실성\\n4. 기밀성"

test_answer1 = "답변: 1"
test_answer2 = "답변: 4"

print("📋 FinShibainu 형태:")
choices1 = extract_choices_from_question(test_question1)
print(f"추출된 선택지: {choices1}")
enhanced1 = enhance_mcqa_answer(test_question1, test_answer1)
print(f"개선된 답변: {enhanced1}")

print("\n📋 CyberMetric/SecBench 형태:")
choices2 = extract_choices_from_question(test_question2)
print(f"추출된 선택지: {choices2}")
enhanced2 = enhance_mcqa_answer(test_question2, test_answer2)
print(f"개선된 답변: {enhanced2}")
print()

# 각 파일 처리
for input_file, output_file in mcqa_files:
    if os.path.exists(input_file):
        try:
            process_mcqa_file(input_file, output_file)
        except Exception as e:
            print(f"❌ {input_file} 처리 중 오류: {e}")
    else:
        print(f"⚠️ 파일 없음: {input_file}")

print("\n🎉 모든 MCQA 파일 처리 완료!")
print("\n📁 생성된 개선된 파일들:")
for input_file, output_file in mcqa_files:
    if os.path.exists(output_file):
        print(f"✅ {output_file}")
    else:
        print(f"❌ {output_file} (생성 실패)")

print("\n💡 QLora 학습 시 mcqa_enhanced.csv 파일들을 사용하세요!")

=== MCQA 답변 형태 개선 시작 (수정된 버전) ===
🗑️ 기존 파일 삭제: ../data/CyberMetric/mcqa_enhanced.csv
🗑️ 기존 파일 삭제: ../data/SecBench/mcqa_enhanced.csv
🗑️ 기존 파일 삭제: ../data/FinShibainu/mcqa_enhanced.csv

🧪 함수 테스트:
📋 FinShibainu 형태:
추출된 선택지: {'1': '기존 대출의 만기 연장', '2': '높은 이자율의 혜택', '3': '조건에 따른 자동 관리', '4': '기본계좌와 연동'}
개선된 답변: 답변: 1, 기존 대출의 만기 연장

📋 CyberMetric/SecBench 형태:
추출된 선택지: {'1': '가용성', '2': '인증', '3': '성실성', '4': '기밀성'}
개선된 답변: 답변: 4, 기밀성

📝 ../data/CyberMetric/mcqa.csv 처리 중...
   총 10180개 항목 발견


답변 개선: 100%|██████████| 10180/10180 [00:00<00:00, 20202.50it/s]


✅ 처리 완료: ../data/CyberMetric/mcqa_enhanced.csv
   10180개 항목이 저장됨

📋 변경 예시:
   원본: 답변: 4
   개선: 답변: 4, 기밀성

   원본: 답변: 3
   개선: 답변: 3, 다중 인증 인증

   원본: 답변: 1
   개선: 답변: 1, 평가 목표

📝 ../data/SecBench/mcqa.csv 처리 중...
   총 2730개 항목 발견


답변 개선: 100%|██████████| 2730/2730 [00:00<00:00, 23942.99it/s]

✅ 처리 완료: ../data/SecBench/mcqa_enhanced.csv
   2730개 항목이 저장됨

📋 변경 예시:
   원본: 답변: 2
   개선: 답변: 2, 데이터 최소화 원칙을 설정하고 필요한 개인 데이터 만 수집합니다.

   원본: 답변: 1
   개선: 답변: 1, 언제든지 개인 데이터에 액세스, 수정 또는 삭제 요청

   원본: 답변: 3
   개선: 답변: 3, 데이터 보안 보호의 원칙

📝 ../data/FinShibainu/mcqa.csv 처리 중...


   총 42463개 항목 발견


답변 개선:   0%|          | 0/42463 [00:00<?, ?it/s]

답변 개선: 100%|██████████| 42463/42463 [00:02<00:00, 19981.69it/s]


✅ 처리 완료: ../data/FinShibainu/mcqa_enhanced.csv
   42463개 항목이 저장됨

📋 변경 예시:
   원본: 답변: 1
   개선: 답변: 1, 기존 대출의 만기 연장

   원본: 답변: 4
   개선: 답변: 4, ELW의 손실은 최초투자금액으로 제한되기 때문에

   원본: 답변: 3
   개선: 답변: 3, 라틴어


🎉 모든 MCQA 파일 처리 완료!

📁 생성된 개선된 파일들:
✅ ../data/CyberMetric/mcqa_enhanced.csv
✅ ../data/SecBench/mcqa_enhanced.csv
✅ ../data/FinShibainu/mcqa_enhanced.csv

💡 QLora 학습 시 mcqa_enhanced.csv 파일들을 사용하세요!
